In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

/home/varad/snap/snapd-desktop-integration/83/Desktop/Falcon/falconllmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name =  "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.10s/it]


In [3]:
from peft import PeftModel

# load perf model with new adapters
ft_model = PeftModel.from_pretrained(
    base_model,
    "outputs-instruct",
)
ft_model = ft_model.merge_and_unload()

In [4]:
def generate_using_string(example_text):


  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = ft_model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 5)

  print("Answer:")
  answer = tokenizer.decode(output[0], skip_special_tokens=True)
  answer = answer.splitlines()[1]
  answer = answer.split('.')[0] + answer.split('.')[1]

  
  print(answer)
  
  print()

In [4]:
from datasets import load_dataset

data = load_dataset("json", data_files="../Dataset/data_test.json")


In [5]:
len(data['train'])

43

In [6]:
import json
with open('answers.json', 'r') as file:
    ans = json.load(file)

In [7]:
def ft_generate(index):

  example_text = data['train'][index]['question']
  correct_answer = data['train'][index]['answer']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = ft_model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  print("Answer Fine Tuned:")
  answer = tokenizer.decode(output[0], skip_special_tokens=True)
  answer = answer.split('\n')[1]
  answer = answer.split('.')
  if(len(answer) >= 2):
    answer = answer[0] + answer[1]
  else:
    answer = answer[0]
  print(answer)

  
  
  print("Best Answer:")
  print(correct_answer)


  ans[index]['ift_answer'] = answer
  print("Best Answer:")
  print(correct_answer)

  print()

In [8]:
for i in range(43):
    ft_generate(i)


/home/varad/snap/snapd-desktop-integration/83/Desktop/Falcon/falconllmenv/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Which mode of transport is advisable for traveling during traffic hours?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer Fine Tuned:
For traveling during traffic hours, it's advisable to use public transport options like buses and metro lines These services have fewer delays and you can arrive at your destinations faster
Best Answer:
The Delhi Metro is a reliable option for traveling during rush hours, as it's not affected by road traffic. Auto-rickshaws and taxis might get stuck in traffic.
Best Answer:
The Delhi Metro is a reliable option for traveling during rush hours, as it's not affected by road traffic. Auto-rickshaws and taxis might get stuck in traffic.

Question:
What are a few well-known markets worth visiting?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

Answer Fine Tuned:
To minimize crowds and have a peaceful visit, consider visiting the attraction early in the morning when it's less crowded, choosing alternative less popular routes, and avoiding peak hours Make an advance booking of the attraction to secure a convenient location
Best Answer:
Minimize crowds and enhance your experience at renowned Delhi attractions by planning your visits early in the morning, right after they open, or by choosing weekdays for your outings.
Best Answer:
Minimize crowds and enhance your experience at renowned Delhi attractions by planning your visits early in the morning, right after they open, or by choosing weekdays for your outings.

Question:
What's the best approach to beat the crowds and fully enjoy Delhi's popular attractions?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer Fine Tuned:
To beat crowds and fully enjoy Delhi's popular attractions, consider visiting them early in the morning just after they open, avoiding peak hours Avoid weekends when crowds are denser
Best Answer:
To beat the crowds and fully enjoy Delhi's popular attractions, plan your visits for the early morning when they open or opt for weekday excursions.
Best Answer:
To beat the crowds and fully enjoy Delhi's popular attractions, plan your visits for the early morning when they open or opt for weekday excursions.

Question:
For traveling in city during busy rush hours, which mode of transportation is the most reliable?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer Fine Tuned:
In city centers, the metro systems are generally reliable and efficient However, when traveling to specific destinations, taxis or auto-rickshaws are a viable option
Best Answer:
To ensure reliability while traveling during busy rush hours in Delhi, it's advisable to choose the Delhi Metro.
Best Answer:
To ensure reliability while traveling during busy rush hours in Delhi, it's advisable to choose the Delhi Metro.

Question:
How can I book a taxi?
Answer Fine Tuned:
You can book a taxi either through a mobile app, by calling a taxi number, or by directly entering the taxi at the airport terminals The options for payment via credit/debit card or cash are available and preferred by many taxi drivers
Best Answer:
You can book a taxi through various ride-hailing apps like Uber and Ola. These apps provide convenient options for booking cabs with upfront pricing and multiple vehicle choices.
Best Answer:
You can book a taxi through various ride-hailing apps like Uber and O

In [9]:
len(ans)

43

In [12]:
import json
with open('answers.json', 'w') as json_file:
    json.dump(ans, json_file)